In [1]:
import tensorflow as tf
import numpy as np

# tf.enable_eager_execution()
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
impression_embed_size = 50 # 853,540개 
poi_embed_size = 50      # 13,352개
filter_embed_size = 50   # 208개
platform_embed_size = 5       # 55개
city_embed_size = 50          # 34,752개
sort_embed_size = 50 #interaction 빼고 one-hot
device_embed_size = 3 # mobile, desktop, tablet one-hot

rnn_input_size = impression_embed_size + 5 # action 5개로 통합 후 one-hot 

#LSTM
hidden_state_size = 64

In [3]:
# Parameters
learning_rate = 0.01
seq_length = 10
num_epoch = 10
batch_size = 1

input_size = 100 # TBD

In [4]:
import pandas as pd
import functions as f
from collections import Counter

In [5]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')
meta_df = pd.read_csv('./item_metadata.csv')
submit_df = pd.read_csv('./submission_popular.csv')

popular_df = f.get_popularity(train_df)

In [6]:
poi_idx = np.load('./npy/poi_names.npy', allow_pickle=True)
poi_idx = list(poi_idx)
impressions_idx = np.load('./npy/impressions_index.npy', allow_pickle=True)
impressions_idx = list(impressions_idx)
city_idx = np.load('./npy/city_names.npy', allow_pickle=True)
city_idx = list(city_idx)
platform_idx = np.load('./npy/platform_names.npy', allow_pickle=True)
platform_idx = list(platform_idx)
filter_idx = np.load('./npy/filter_merged.npy', allow_pickle=True)
filter_idx = list(filter_idx)
action_idx = np.load('./npy/action_type_names.npy', allow_pickle=True)
action_idx = list(action_idx)

sort_order_idx = np.load('./npy/sorting_names.npy', allow_pickle=True)
sort_order_idx = list(sort_order_idx)
device_idx = np.load('./npy/device_names.npy', allow_pickle=True)
device_idx = list(device_idx)
'''
poi_embedding = tf.get_variable("poi_embedding", [len(poi_idx), poi_embed_size])
impression_embedding = tf.get_variable("impression_embedding", [len(impressions_idx), impression_embed_size])
city_embedding = tf.get_variable("city_embedding", [len(city_idx), city_embed_size])
platform_embedding = tf.get_variable("platform_embedding", [len(platform_idx), platform_embed_size])
filter_embedding = tf.get_variable("filter_embedding", [len(filter_idx), filter_embed_size])
'''

'\npoi_embedding = tf.get_variable("poi_embedding", [len(poi_idx), poi_embed_size])\nimpression_embedding = tf.get_variable("impression_embedding", [len(impressions_idx), impression_embed_size])\ncity_embedding = tf.get_variable("city_embedding", [len(city_idx), city_embed_size])\nplatform_embedding = tf.get_variable("platform_embedding", [len(platform_idx), platform_embed_size])\nfilter_embedding = tf.get_variable("filter_embedding", [len(filter_idx), filter_embed_size])\n'

In [7]:
train_label = train_df[(train_df["action_type"] == "clickout item")]     #train_label : (1,586,586, 12)
batch_indexes = np.random.permutation(len(train_label))

In [8]:
def embedding_function(domain, key):
    
    embedded_vector = tf.zeros([impression_embed_size])
    if domain == action_idx[9]:  #'poi'
        if key in poi_idx:
            idx = poi_idx.index(key)
        else:
            idx = -1
        embedded_vector = tf.nn.embedding_lookup(poi_embedding, idx)
        
    elif (domain == action_idx[1])|(domain == action_idx[3])|(domain == action_idx[4])|(domain == action_idx[5])\
    |(domain == action_idx[6])|(domain == action_idx[8]):  #impression
        if key in impressions_idx:
            idx = impressions_idx.index(key)
        else:
            idx = -1
        embedded_vector = tf.nn.embedding_lookup(impression_embedding, idx)
    
    elif domain == action_idx[7]: # search for destination, 'city'
        if key in city_idx:
            idx = city_idx.index(key)
        else:
            idx = -1
        embedded_vector = tf.nn.embedding_lookup(city_embedding, idx)

    elif domain == 'platform':
        if key in platform_idx:
            idx = platform_idx.index(key)
        else:
            idx = -1
        embedded_vector = tf.nn.embedding_lookup(platform_embedding, idx)
        
    elif domain ==action_idx[2]:  #'filter'
        if key in filter_idx:
            idx = filter_idx.index(key)
        else:
            idx = -1
        embedded_vector = tf.nn.embedding_lookup(filter_embedding, idx)
        
    elif domain ==action_idx[0]:  #sorting
        embedded_vector = np.zeros([sort_embed_size])
        embedded_vector[sort_order_idx.index(key)] = 1
        embedded_vector = tf.convert_to_tensor(embedded_vector, dtype=tf.float32)
        
    elif domain =='device':
        embedded_vector = np.zeros([device_embed_size])
#         embedded_vector[device_idx.index(key)] = 1
        embedded_vector = tf.convert_to_tensor(embedded_vector, dtype=tf.float32)
        
    return embedded_vector

In [9]:
action_idx

['change of sort order',
 'clickout item',
 'filter selection',
 'interaction item deals',
 'interaction item image',
 'interaction item info',
 'interaction item rating',
 'search for destination',
 'search for item',
 'search for poi']

In [10]:
def action_embedding(action):
    if action == action_idx[0]:
        embed = tf.one_hot(0, 5)
    elif action == action_idx[2]:
        embed = tf.one_hot(1, 5)
    elif action == action_idx[7]:
        embed = tf.one_hot(2, 5)
    elif action == action_idx[9]:
        embed = tf.one_hot(3, 5)
    else:
        embed = tf.one_hot(4, 5)
    return embed

In [11]:
INDEX_user_id =0                                                 
INDEX_session_id =1                                            
INDEX_timestamp =2                                                
INDEX_step        =3                                                       
INDEX_action_type =4                                       
INDEX_reference   =5                                               
INDEX_platform    =6                                                
INDEX_city        =7                                          
INDEX_device      =8                                                 
INDEX_current_filters=9                                                
INDEX_impressions    =10   
INDEX_prices         =11    

In [12]:
def getSequenceData(act_list, ref_list, seq_length):
    embed_total = tf.zeros([rnn_input_size])  #추후 1 -> batch size 변경
    embed_total = tf.reshape(embed_total, [1, rnn_input_size])
    for step in range(np.shape(act_list)[1]): 
        embed_ref = embedding_function(act_list[step], ref_list[step])
        embed_action = action_embedding(act_list[step])
        
        embed_single_action = tf.concat((embed_action, embed_ref), axis=0)
        embed_total = tf.concat((embed_total, tf.reshape(embed_single_action, [1,55])), axis=0)
        
    return embed_total[1:,:]

In [13]:
def getActionList(user_label, seq_length):
    step = user_label['step']
    if step > seq_length+1:
        step = seq_length+1
    label_point = user_label.name
    history_list = train_df[label_point-step+1:label_point]
    
    act_list=[]
    ref_list=[]
    for idx in range(len(history_list)):
        act_list.append(history_list.iloc[idx]['action_type'])
        ref_list.append(history_list.iloc[idx]['reference'])
    if len(act_list) < seq_length:
        act_list = np.concatenate((np.zeros([seq_length-len(act_list)]), act_list))
        ref_list = np.concatenate((np.zeros([seq_length-len(ref_list)]), ref_list))
    
    return np.reshape(act_list, [-1, seq_length]), np.reshape(ref_list, [-1, seq_length])

In [14]:
def recurrent_neural_network_model(Act_list, Ref_list):
    
    XX = getSequenceData(Act_list, Ref_list, seq_length)
    XX = tf.reshape(XX, [batch_size,seq_length, rnn_input_size])
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_state_size)    
    _ , states = tf.nn.dynamic_rnn(lstm_cell, XX, dtype=tf.float32)   
    
    return states

In [79]:
# Define the neural network
tf.reset_default_graph()
Act_list = tf.placeholder(tf.string, [None, seq_length])
Ref_list = tf.placeholder(tf.string, [None, seq_length])

Train_Label_df = tf.placeholder(tf.string, [None, 12])
Target_impression = tf.placeholder(tf.int32, [None, 1])
Y_label = tf.placeholder(tf.int32, [None, 1])

poi_embedding = tf.get_variable("poi_embedding", [len(poi_idx), poi_embed_size])
impression_embedding = tf.get_variable("impression_embedding", [len(impressions_idx), impression_embed_size])
city_embedding = tf.get_variable("city_embedding", [len(city_idx), city_embed_size])
platform_embedding = tf.get_variable("platform_embedding", [len(platform_idx), platform_embed_size])
filter_embedding = tf.get_variable("filter_embedding", [len(filter_idx), filter_embed_size])

In [80]:
def dense_net(Act_list, Ref_list, train_label, target_impression):
    
    XX = getSequenceData(Act_list, Ref_list, seq_length)
    XX = tf.reshape(XX, [batch_size,seq_length, rnn_input_size])
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_state_size)    
    _ , states = tf.nn.dynamic_rnn(lstm_cell, XX, dtype=tf.float32)   
    context = tf.reshape(states[0], [64])
        #platform
    
    if target_impression in impressions_idx:
        idx = impressions_idx.index(key)
    else:
        idx = -1
    target_impression_embed = tf.nn.embedding_lookup(impression_embedding, idx)
    
        
    #platform_embed = embedding_function('platform', train_label[-1,INDEX_platform])
    key = train_label[-1,INDEX_platform]
    if key in platform_idx:
        idx = platform_idx.index(key)
    else:
        idx = 0
    platform_embed = tf.nn.embedding_lookup(platform_embedding, idx)
    #city, search for destination 해서 검색해봤던 city들
    #city_embed = embedding_function('city', train_label[-1,INDEX_city])
    key = train_label[-1,INDEX_city]
    if key in city_idx:
        idx = city_idx.index(key)
    else:
        idx = 0
    city_embed = tf.nn.embedding_lookup(city_embedding, idx)
    #device
    #device_embed = embedding_function('device', train_label[-1,INDEX_device])
    key = train_label[-1,INDEX_device]
    device_embed = np.zeros([device_embed_size])
#         embedded_vector[device_idx.index(key)] = 1
    device_embed= tf.convert_to_tensor(device_embed, dtype=tf.float32)
    #current_filter
    
    filter_b = train_label[-1,INDEX_current_filters]
    if pd.isna(filter_b):
        final_embed_current_filter = tf.zeros(filter_embed_size)
    else:
        final_embed_current_filter = tf.zeros(filter_embed_size)
#         filters_b = f.string_to_array(filter_b)
#         final_embed_current_filter = getWeightedAverage('filter', filters_b , filter_embed_size)
    
#     print("context", np.shape(context))
#     print("platform_embed", np.shape(platform_embed))
#     print("city_embed", np.shape(city_embed))
#     print("device_embed", np.shape(device_embed))
    
    XX_input= tf.concat((context, platform_embed, city_embed, device_embed, target_impression_embed),axis=0)
    
    XX_input = tf.reshape(XX_input, [-1, 172])
    layer_1 = tf.layers.dense(XX_input, 10,activation=tf.nn.relu)
    layer_2 = tf.layers.dense(layer_1, 10,activation=tf.nn.relu)
    layer_3 = tf.layers.dense(layer_2, 10, activation=tf.nn.relu)
    out_layer = tf.layers.dense(layer_3, 1, activation=tf.nn.sigmoid)
    return out_layer

In [38]:
output = dense_net(Act_list, Ref_list, Train_Label_df)

In [39]:
loss = tf.losses.sigmoid_cross_entropy(Y_label, output)

Instructions for updating:
Use tf.cast instead.


In [41]:
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

Instructions for updating:
Use tf.cast instead.


In [42]:
sess = tf.Session()

In [43]:
sess.run(tf.global_variables_initializer())

In [88]:
label_df = None
batch_size = 10
for k in range(2):
    idx = batch_indexes[k*batch_size:(k+1)*batch_size]
    label_df = train_label.iloc[idx]
    for i in range(2):  # batch size
        user_label = label_df.iloc[i]
        train_label_batch = user_label["reference"]  # 클릭한 reference number
        train_candidates = user_label['impressions']
        train_candidates = f.string_to_array(train_candidates)   
        
        candidate_prices = user_label['prices']
        candidate_prices = f.string_to_array(candidate_prices)
        candidate_prices = np.array(candidate_prices).astype(int)
        
        label_index = train_candidates.index(train_label_batch)
        labels = np.zeros([len(train_candidates)],dtype=int)
        labels[label_index]=1
        
        act_list, ref_list = getActionList(user_label, seq_length)
        user_label = user_label.fillna('Nope')
        users = np.array(user_label)
        users= np.reshape(users,[-1,12])     
        
        for h in range(len(train_candidates)):
            _, loss_val = sess.run([train, loss], feed_dict={Act_list: act_list, Ref_list: ref_list, Train_Label_df: users,\
                                                             Y_label:labels[h], Target_impression: int(train_candidates[h])})
            print("loss : ", loss_val)
        
        
        
            
            
        
        

TypeError: Cannot interpret feed_dict key as Tensor: Tensor Tensor("Placeholder:0", shape=(?, 10), dtype=string) is not an element of this graph.

In [89]:
k=0
i=0

In [90]:
idx = batch_indexes[k*batch_size:(k+1)*batch_size]
label_df = train_label.iloc[idx]

In [91]:
        user_label = label_df.iloc[i]
        train_label_batch = user_label["reference"]  # 클릭한 reference number
        train_candidates = user_label['impressions']
        train_candidates = f.string_to_array(train_candidates)   
        
        candidate_prices = user_label['prices']
        candidate_prices = f.string_to_array(candidate_prices)
        candidate_prices = np.array(candidate_prices).astype(int)
        
        label_index = train_candidates.index(train_label_batch)
        labels = np.zeros([len(train_candidates)],dtype=int)
        labels[label_index]=1
        
        act_list, ref_list = getActionList(user_label, seq_length)
        user_label = user_label.fillna('Nope')
        users = np.array(user_label)
        users= np.reshape(users,[-1,12])   

In [92]:
train_label_batch

'43835'

In [93]:
act_list

array([['0.0', '0.0', '0.0', '0.0', 'filter selection', 'clickout item',
        'filter selection', 'filter selection', 'clickout item',
        'clickout item']], dtype='<U32')

In [94]:
ref_list

array([['0.0', '0.0', '0.0', '0.0', 'Swimming Pool (Combined Filter)',
        '44573', 'Swimming Pool (Combined Filter)', 'Car Park', '44644',
        '44644']], dtype='<U32')

In [95]:
users

array([['NA5FFXBGTX5P', '7ee3de02b8f98', 1541354193, 7, 'clickout item',
        '43835', 'CA', 'London, Canada', 'mobile', 'Nope',
        '43835|44389|1455805|43373|43861|43552|43725|347366|44609|941294|397676|843481|43606|44603|43610|44644|43580|43924|44360|44078|1181148|10572410|43684|44701|44124',
        '85|73|96|54|86|113|93|79|94|48|83|80|113|44|57|84|87|57|94|80|83|73|70|50|86']],
      dtype=object)